Naive Bayes Classifier

In this project we try to classify a group of comments using naive bayes classifier. Using this classification, we will determine whether a comment is negative or positive. 
To do this, we find the probability of different features belonging to different classes. Using these probabilities, we find the probabilities of a new data belonging to each class. whichever class that has the highest probability, we choose as the class the new data belongs to.

In [1266]:
import numpy as np
import csv
import pandas as pd
from numpy import random
import math
from tabulate import tabulate
num_of_classes=2

First we get the training data to train our model with.

In [1267]:
comment_train = pd.read_csv('comment_train.csv').to_numpy()

Part1. In this part, first we need to preprocess the data for the next parts. These preprocesses can be helpful or can reduce our accuracy. In what follows, we will see which ones help with our classification and decide which ones we will use on test data. 
We do these preprocesses using hazm.

In [1268]:
from __future__ import unicode_literals
from hazm import *

In [1269]:
    comment_train1=comment_train.copy()
    comment_train2=comment_train.copy()
    comment_train3=comment_train.copy()
    comment_train4=comment_train.copy()

In [1270]:
def preprocess_all(comment_train1):
    normalizer = Normalizer()
    stemmer = Stemmer()
    lemmatizer = Lemmatizer()
    for i in range(len(comment_train1)):
        for j in range(len(comment_train1[i])):
            comment_train1[i][j]=normalizer.normalize(comment_train1[i][j])
            comment_train1[i][j]=word_tokenize(comment_train1[i][j])
            for k in range(len(comment_train1[i][j])):
                comment_train1[i][j][k]=stemmer.stem(comment_train1[i][j][k])
                comment_train1[i][j][k]=lemmatizer.lemmatize(comment_train1[i][j][k])
    return comment_train1


In [1271]:
def preprocess_withjustlem(comment_train2):
    normalizer = Normalizer()
    lemmatizer = Lemmatizer()
    for i in range(len(comment_train2)):
        for j in range(len(comment_train2[i])):
            comment_train2[i][j]=normalizer.normalize(comment_train2[i][j])
            comment_train2[i][j]=word_tokenize(comment_train2[i][j])
            for k in range(len(comment_train2[i][j])):
                comment_train2[i][j][k]=lemmatizer.lemmatize(comment_train2[i][j][k])
    return comment_train2

In [1272]:
def preprocess_withjuststem(comment_train3):
    normalizer = Normalizer()
    stemmer = Stemmer()
    for i in range(len(comment_train3)):
        for j in range(len(comment_train3[i])):
            comment_train3[i][j]=normalizer.normalize(comment_train3[i][j])
            comment_train3[i][j]=word_tokenize(comment_train3[i][j])
            for k in range(len(comment_train3[i][j])):
                comment_train3[i][j][k]=stemmer.stem(comment_train3[i][j][k])
    return comment_train3

In [1273]:
def preprocess_nopre(comment_train4):
    normalizer = Normalizer()
    for i in range(len(comment_train4)):
        for j in range(len(comment_train4[i])):
            comment_train4[i][j]=normalizer.normalize(comment_train4[i][j])
            comment_train4[i][j]=word_tokenize(comment_train4[i][j])
    return comment_train4

In [1274]:
comment_train_nopre=preprocess_nopre(comment_train4)

In [1275]:
comment_train_nopre[117][1]

['خیلی',
 'بی',
 'کیفیت',
 'هست',
 'و',
 'سایزهاش',
 'خیلی',
 'کوچیک',
 'و',
 'بدون',
 'کاربرد',
 'هست',
 'واقعا',
 'من',
 'توی',
 'شگفت',
 'انگیز',
 'خریدم',
 'ولی',
 'تو',
 'قیمت',
 'شگفت',
 'انگیز',
 'هم',
 'ارزش',
 'خرید',
 'نداره',
 '.']

In [1276]:
comment_train_justlem=preprocess_withjustlem(comment_train2)

In [1277]:
comment_train_justlem[117][1]

['خیلی',
 'بی',
 'کیفیت',
 '#هست',
 'و',
 'سایز',
 'خیلی',
 'کوچیک',
 'و',
 'بدون',
 'کاربرد',
 '#هست',
 'واقعا',
 'من',
 'تو',
 'شگفت',
 'انگیز',
 'خرید#خر',
 'ولی',
 'تو',
 'قیمت',
 'شگفت',
 'انگیز',
 'هم',
 'ارزش',
 'خرید',
 'نداره',
 '.']

In [1278]:
comment_train_juststem=preprocess_withjuststem(comment_train3)

In [1279]:
comment_train_juststem[117][1]

['خیل',
 'ب',
 'کیف',
 'هس',
 'و',
 'سایز',
 'خیل',
 'کوچیک',
 'و',
 'بدون',
 'کاربرد',
 'هس',
 'واقعا',
 'من',
 'تو',
 'شگف',
 'انگیز',
 'خرید',
 'ول',
 'تو',
 'قیم',
 'شگف',
 'انگیز',
 'ه',
 'ارز',
 'خرید',
 'نداره',
 '.']

In [1280]:
comment_train = preprocess_all(comment_train1)

In [1281]:
comment_train[117][1]

['خیل',
 'ب',
 'کیف',
 'هس',
 'و',
 'سایز',
 'خیل',
 'کوچیک',
 'و',
 'بدون',
 'کاربرد',
 'هس',
 'واقعا',
 'من',
 'تو',
 'شگف',
 'انگیز',
 'خرید',
 'ول',
 'تو',
 'قیم',
 'شگف',
 'انگیز',
 'ه',
 'ارز',
 'خرید',
 'نداره',
 '.']

q1. Stemming helps us find a word despite it being plural or singular. In theory, this helps of prevent producing a lot of features that can make our classification slower. It also helps us have better accuracy. 
Lemmatization helps us find the root of a verb. In theory, this also help us with producing less features and having better accuracy by providing better probabilities.
In what follows we will we whether this will be true. 

Part 2. 

In this part, we use the second column for our calculations, since it has more words that can help us predict its category better.

q2. Evidence are the information we already have from our training data. Since evidence is the same for all datas, we do not need to calculate it. It will not change the following comparison's results.
Likelihood show us the probability of the test data blonging to a particular class. Since we are using naive bayes, this probability is the multiplications of the probabilities of each feature blonging to a class. 
Prior shows us the probability of a data belonging to a class based on the training data and without condidering the data's features.
Post shows us the probability of a data belonging to a class based on the data's features and our classification model.


In [1282]:
def class_numbers_in_features_table(s):
    if s==['recommended']:
        return 1
    else :
        return 2

In [1283]:
def cal_feature_count(comment_train,num_of_classes):
    features=[]
    for i in range(num_of_classes+1):
        features.append([])
    tot_num=len(comment_train)
    tot_recom=0
    tot_not=0
    for i in range(len(comment_train)):
        if comment_train[i][2]==['recommended']:
            tot_recom+=1
        else:
            tot_not+=1
        for j in range(len(comment_train[i][1])):
            if comment_train[i][1][j] in features[0]:
                features[class_numbers_in_features_table(comment_train[i][2])][features[0].index(comment_train[i][1][j])]+=1
            else:
                features[0].append(comment_train[i][1][j])
                features[1].append(0)
                features[2].append(0)
                features[class_numbers_in_features_table(comment_train[i][2])][features[0].index(comment_train[i][1][j])]+=1
    return features


In [1284]:
features=cal_feature_count(comment_train,num_of_classes)

In [1285]:
features[1][1]

63

q3. Since in naive bayes we multiply the probality of features in a class, if one of these probalities is zero, then the whole probality becomes zero and other features are ignored. 

q4. To prevent such happening, we use additive smoothing. We can add a constant like one to all the numbers of repeated features in a class. Hence, the features that does not exist in one class will not make the result zero and this will not change the outcome because adding the same constant to all the features does not change the result of comparison.

Part 3. 

q5. Precision shows us the number of detected recommnended to all the ones we assumed to be recommnended and recal shows us the number of detected recommnended to all the recommnendeds. 
If the precision is high, we know that among the ones we detected as recommnended most of them were correct, but there is a chance that there are a lot of recommnended ones that we did not detect and this will not be shown in the presicion. 
In contrast, if recall is high, we know that we detected a lot of recommnendeds but there is also the chance that we got a lot of not recommnendeds as recommnendeds, which again does not show in the recall.
Hence, we need both precision and reacall to determine the quality of classification. If both of these numbers are high, it means that we were able to detect most of the recommnendeds and did not get a lot of wrongs.

q6. Using this mean, the smaller the number is, the more its weight in calculating the mean becomes. If one of the numbers is very small, it effects the mean more than the other number does. In contrast, if we had used normal mean, if one of the numbers was very small but the other one very big, we could not have known that one of them was small which could bring the quality of classification down. Using this mean, the result will always show if one of the numbers is vert small, hence, we can accurately analyze our classification. 

In this part, first we get the test data, and then we decide based on the probabilities that we have, wether the data is recommended or not. Then we compare the predected results to the actual result to see how our classification works.

In [1286]:
comment_test = pd.read_csv('comment_test.csv').to_numpy()

In [1287]:
comment_test[119][1]

'برد بالایی نداره کیفیت صداش معمولیه برای تایم تمرین و ورزش مناسبه از روی گوش نمیفته  ظریف نیست میان نمونه های هم رده خودش خیلی خوب نیست'

In [1288]:
    comment_test2=comment_test.copy()
    comment_test3=comment_test.copy()
    comment_test4=comment_test.copy()

In [1289]:
comment_test=preprocess_all(comment_test)

In [1290]:
comment_test[119][1]

['برد',
 'بالا',
 'نداره',
 'کیف',
 'صدا',
 'معمولیه',
 'برا',
 'تا',
 'تمرین',
 'و',
 'ورز',
 'مناسبه',
 'از',
 'رو',
 'گو',
 'نمیفته',
 'ظریف',
 'نیس',
 'م',
 'نمونه',
 'ه',
 'رده',
 'خود',
 'خیل',
 'خوب',
 'نیس']

In [1291]:
comment_test_justlem=preprocess_withjustlem(comment_test2)
comment_test_juststem=preprocess_withjuststem(comment_test3)
comment_test_nopre=preprocess_nopre(comment_test4)

In [1292]:
test_real_outputs=[]
for i in range(len(comment_test)):
    test_real_outputs.append(class_numbers_in_features_table(comment_test[i][2]))

In [1293]:
def cal_denum(de_features):
    de_num=[0,0]
    for i in range(len(de_features[0])):
        for j in range(len(de_num)):
            de_num[j]+=de_features[j+1][i]
    return de_num
# In this fuction we calculate the denum of the probability we the sum of all the words in that class

In [1294]:
def smoothing(to_be_smoothed_features):
    for i in range(len(to_be_smoothed_features[0])):
        to_be_smoothed_features[1][i]+=1
        to_be_smoothed_features[2][i]+=1
    return to_be_smoothed_features


In [1295]:
def cal_prob_of_features(train_features_num,denums):
    for i in range(len(train_features_num)-1):
        for j in range(len(train_features_num[i])):
            train_features_num[i+1][j]=int(train_features_num[i+1][j])/int(denums[i])
    return train_features_num


In [1296]:
def cal_prob_of_single_testdata_log(test_features, prob_of_feaures):
    prob_of_single_data=[0 ,0]
    for i in range(len(test_features)):
        if test_features[i] in prob_of_feaures[0]:
            index=prob_of_feaures[0].index(test_features[i])
            prob_of_single_data[0]+=math.log(prob_of_feaures[1][index])
            prob_of_single_data[1]+=math.log(prob_of_feaures[2][index])
    return prob_of_single_data  #returns 2 numbers


In [1297]:
def cal_prob_of_single_testdata_mul(test_features, prob_of_feaures):
    prob_of_single_data=[1 ,1]
    for i in range(len(test_features)):
        if test_features[i] in prob_of_feaures[0]:
            index=prob_of_feaures[0].index(test_features[i])
            prob_of_single_data[0]*=prob_of_feaures[1][index]
            prob_of_single_data[1]*=prob_of_feaures[2][index]
    return prob_of_single_data  #returns 2 numbers

In [1298]:
def cal_output_withsmooth(test, train_features):
    train_features = smoothing(train_features)
    denums = cal_denum(train_features)
    prob_of_feaures = cal_prob_of_features(train_features,denums)
    predicted_out=[]
    for i in range(len(test)):
        single_test_prob=cal_prob_of_single_testdata_log(test[i][1], prob_of_feaures)
        if single_test_prob[0]>single_test_prob[1]:
            predicted_out.append(1)
        else:
            predicted_out.append(2)
    return predicted_out


In [1299]:
calculated_out_test = cal_output_withsmooth(comment_test, features)
out_to_see=calculated_out_test

In [1300]:
def cal_ac(calculated_out_test,test_real_outputs):
    correct=0
    for i in range(len(calculated_out_test)):
        if calculated_out_test[i]==test_real_outputs[i]:
            correct+=1
    return(correct/len(calculated_out_test))

In [1301]:
accuracy= cal_ac(calculated_out_test,test_real_outputs)

This accuracy is for both stemming and lemmatization were used.
We also had smoothing.

In [1302]:
accuracy

0.9025

More on 1.

In [1303]:
features=cal_feature_count(comment_train_justlem,num_of_classes)
calculated_out_test_lem = cal_output_withsmooth(comment_test_justlem, features)
accuracy_lem= cal_ac(calculated_out_test_lem,test_real_outputs)
print('accuracy_lem',accuracy_lem)

accuracy_lem 0.9125


In [1304]:
features=cal_feature_count(comment_train_juststem,num_of_classes)
calculated_out_test_stem = cal_output_withsmooth(comment_test_juststem, features)
accuracy_stem= cal_ac(calculated_out_test_stem,test_real_outputs)
print('accuracy_stem',accuracy_stem)

accuracy_stem 0.91


In [1305]:
features=cal_feature_count(comment_train_nopre,num_of_classes)
calculated_out_test_nopre = cal_output_withsmooth(comment_test_nopre, features)
accuracy_nopre= cal_ac(calculated_out_test_nopre,test_real_outputs)
print('accuracy_nopre',accuracy_nopre)

accuracy_nopre 0.91375


continue 1. With having the results, we can see that just doing normalization and tokenization we have the best accuracy compared to when we use stemming and lamminizaion. This is because 

In [1306]:
def precision(out_test,test_real_outputs):
    correct_detected_recommended=0
    all_detected_recommended=0
    for i in range(len(out_test)):
        if out_test[i]==1:
            all_detected_recommended+=1
        if (out_test[i]==test_real_outputs[i]) and (out_test[i]==1):
            correct_detected_recommended+=1
    return(correct_detected_recommended/all_detected_recommended)

In [1307]:
def recall(_out_test,test_real_outputs):
    correct_detected_recommended=0
    all_recommended=0
    for i in range(len(_out_test)):
        if test_real_outputs[i]==1:
            all_recommended+=1
        if (_out_test[i]==test_real_outputs[i]) and (_out_test[i]==1):
            correct_detected_recommended+=1
    return (correct_detected_recommended/all_recommended)

In [1308]:
def f1(a,b):
    return (2*a*b)/(a+b)

In [1309]:
print('accuracy with pre with smooth',accuracy)
preps=precision(calculated_out_test,test_real_outputs)
print(preps)
reps=recall(calculated_out_test,test_real_outputs)
print(reps)
f1_withpre_withsmooth=f1(reps,preps)
print('f1_withpre_withsmooth',f1_withs_withpre_withsmooth)

accuracy with pre with smooth 0.9025
0.8815165876777251
0.93
f1_withpre_withsmooth 0.8623853211009176


In [1310]:
def cal_output_withoutsmooth(test, train_features):
    denums = cal_denum(train_features)
    prob_of_feaures = cal_prob_of_features(train_features,denums)
    predicted_out=[]
    for i in range(len(test)):
        single_test_prob=cal_prob_of_single_testdata_mul(test[i][1], prob_of_feaures)
        if single_test_prob[0]>single_test_prob[1]:
            predicted_out.append(1)
        else:
            predicted_out.append(2)
    return predicted_out

In [1311]:
features_pre_nosmooth=cal_feature_count(comment_train,num_of_classes)
calculated_out_test_pre_nosmooth = cal_output_withoutsmooth(comment_test, features_pre_nosmooth)
accuracy_withoutsmooth= cal_ac(calculated_out_test_pre_nosmooth,test_real_outputs)

In [1312]:
print('accuracy with pre no smooth',accuracy_withoutsmooth)
prep=precision(calculated_out_test_pre_nosmooth,test_real_outputs)
print(prep)
rep=recall(calculated_out_test_pre_nosmooth,test_real_outputs)
print(rep)
f1_withpre_nosmooth=f1(rep,prep)
print('f1_withpre_withsmooth',f1_withpre_nosmooth)

accuracy with pre no smooth 0.87125
0.8938992042440318
0.8425
f1_withpre_withsmooth 0.8674388674388674


In [1313]:
print('accuracy without pre with smooth',accuracy_nopre)
pres=precision(calculated_out_test_nopre,test_real_outputs)
print(pres)
res=recall(calculated_out_test_nopre,test_real_outputs)
print(res)
f1_withoutpre_withsmooth=f1(res,pres)
print('f1_withoutpre_withsmooth',f1_withoutpre_withsmooth)

accuracy without pre with smooth 0.91375
0.902676399026764
0.9275
f1_withoutpre_withsmooth 0.9149198520345253


In [1314]:
features=cal_feature_count(comment_train_nopre,num_of_classes)
calculated_out_test_nopre_nosmooth = cal_output_withoutsmooth(comment_test_nopre, features)
accuracy_nopre_nosmooth= cal_ac(calculated_out_test_nopre_nosmooth,test_real_outputs)

In [1315]:
print('accuracy without pre without smooth',accuracy_nopre_nosmooth)
pre=precision(calculated_out_test_nopre_nosmooth,test_real_outputs)
print(pre)
re=recall(calculated_out_test_nopre_nosmooth,test_real_outputs)
print(re)
f1_withoutpre_withoutsmooth=f1(re,pre)
print('f1_withoutpre_withoutsmooth',f1_withoutpre_withoutsmooth)

accuracy without pre without smooth 0.86875
0.90633608815427
0.8225
f1_withoutpre_withoutsmooth 0.8623853211009176


q7.

In [1316]:
print('Accuracy Table')
print(tabulate([['with smoothing', str(accuracy),str(accuracy_nopre)], ['without smoothing', str(accuracy_withoutsmooth),str(accuracy_nopre_nosmooth)]], headers=['','with preprocess', 'without preprocess']))

Accuracy Table
                     with preprocess    without preprocess
-----------------  -----------------  --------------------
with smoothing               0.9025                0.91375
without smoothing            0.87125               0.86875


In [1317]:
print('Precision Table')
print(tabulate([['with smoothing', str(preps),str(pres)], ['without smoothing', str(prep),str(pre)]], headers=['','with preprocess', 'without preprocess']))

Precision Table
                     with preprocess    without preprocess
-----------------  -----------------  --------------------
with smoothing              0.881517              0.902676
without smoothing           0.893899              0.906336


In [1318]:
print('Recall Table')
print(tabulate([['with smoothing', str(reps),str(res)], ['without smoothing', str(rep),str(re)]], headers=['','with preprocess', 'without preprocess']))

Recall Table
                     with preprocess    without preprocess
-----------------  -----------------  --------------------
with smoothing                0.93                  0.9275
without smoothing             0.8425                0.8225


In [1319]:
print('F1 Table')
print(tabulate([['with smoothing', str(f1_withs_withpre_withsmooth),str(f1_withoutpre_withsmooth)], ['without smoothing', str(f1_withpre_nosmooth),str(f1_withoutpre_withoutsmooth)]], headers=['','with preprocess', 'without preprocess']))

F1 Table
                     with preprocess    without preprocess
-----------------  -----------------  --------------------
with smoothing              0.862385              0.91492
without smoothing           0.867439              0.862385


q8. Considering accuracy, we can see that smoothing has a very big impact on the result. This is because(as explained before) when a word does not exist in either recommended or not recommended classes, it's probability becomes zero and after calculating the whole probability, it becomes zero and all the other features are ignored. Smoothing removes this problem.We can also see that the processes stemming and lemmitization has not help with the problem.
Considering precision we can see that numbres are not very different from each other but over all not processed data has had better results because not processing helps us keep the exact words of each class which could help us not detect wrong ones, although we may miss a lot of right ones.
Considering recal, we can see processing data and using smoothing creates the best results. Overall smoothing has a great impact on recall because smoothing prevents us from missing right answers, though we may choose a lot of wrong ones.
Considering F1 which considers both precision and recall, we can see no preprocessing and using smoothing is the best solution, which we can see with accuracy too.

q9.

In [1320]:
wrong_indexes=[]
for i in range(len(calculated_out_test)):
    if test_real_outputs[i]!=calculated_out_test[i]:
        wrong_indexes.append(i)

In [1321]:
index=0
print('1:','test_real_outputs',test_real_outputs[wrong_indexes[index]])
print('calculated_out_test',calculated_out_test[wrong_indexes[index]])
print('comment',comment_test[wrong_indexes[index]][1])

1: test_real_outputs 1
calculated_out_test 2
comment ['تازه', 'خرید', 'یه', 'مد', 'کار', 'بکنه', 'مشخص', 'میشه', 'کیف', 'قطعات']


In this example the comment is recommended but we have guessed wrong. This is probably because words like کیفیت  , تاره ,کار ,etc are usually used when we want to say something is not working or does not have a good quality.

In [1322]:
index=1
print('1:','test_real_outputs',test_real_outputs[wrong_indexes[index]])
print('calculated_out_test',calculated_out_test[wrong_indexes[index]])
print('comment',comment_test[wrong_indexes[index]][1])

1: test_real_outputs 1
calculated_out_test 2
comment ['سلا', '،', 'راح', 'شد#شو', 'از', 'کابل', 'شارژ', '،', 'توصیه', 'میشود', 'به', 'شد#شو', '.', 'ارز', 'گوش', 'خود', 'را', 'به', 'شارژ', 'وایرلس', 'مجهز', 'کرد#کن', '.']


In this case also we have guessed a  recommended comment a not recommended one. This seems to be mainly because of using words like    ارز , توصیه, etc.

In [1323]:
index=2
print('1:','test_real_outputs',test_real_outputs[wrong_indexes[index]])
print('calculated_out_test',calculated_out_test[wrong_indexes[index]])
print('comment',comment_test[wrong_indexes[index]][1])

1: test_real_outputs 2
calculated_out_test 1
comment ['من', 'تو', 'تخفیف', 'ویژه', '۵', 'تا', 'خرید', 'و', 'همشون', 'رو', 'ه', 'تس', 'کرد#کن', 'و', 'اینکه', 'میگن', 'دوس', 'که', 'خرابه', 'برا', 'من', 'پ', 'نیومده', 'انتقال', 'فایل', 'ه', 'میشه', 'با', 'کامپیو', 'اون', 'مشکل', 'ندارن', 'شارژ', 'سریع', 'تس', 'کرد#کن', 'اون', 'اوک', 'هس', '(', 'دوس', 'اینکه', 'کابل', 'فقط', 'شار', 'سریع', 'باشه', 'کاف', 'نیس', 'برا', 'اینکه', 'گوشیتون', 'سریع', 'شارژ', 'بشه', ',', 'کلگ', 'شارژ', 'و', 'خود', 'گوش', 'ه', 'باید', 'قابل', 'شارژ', 'سریع', 'رو', 'ه', 'داشته', 'باشه', ')', 'فقط', 'تن', 'مشکل', 'اینکه', 'جنس', 'خود', 'س', 'یک', 'ب', 'کیف', 'هس', 'و', 'امیدوار', 'پاره', 'نشن', 'برا', 'مد', 'عمر', 'فقط', 'کوتاهس', 'بین', '۱', 'تا', '۳', 'ماه', 'ب', 'دوا', 'نداره', 'البته', 'یک', 'یک', 'هفته', 'ه', 'دوا', 'آورد#آور', 'در', 'ضمن', 'این', 'کابل', 'طرح', 'خوب', 'هستن', 'و', 'فابریک', 'سامسونگ', 'نیستن']


 Here we have guessed a not recommended as recommended probabily because of words like خوب  , فابریک,etc.

In [1324]:
index=3
print('1:','test_real_outputs',test_real_outputs[wrong_indexes[index]])
print('calculated_out_test',calculated_out_test[wrong_indexes[index]])
print('comment',comment_test[wrong_indexes[index]][1])

1: test_real_outputs 1
calculated_out_test 2
comment ['فندک', 'قبل', 'مدا', 'فیوز', 'میسوزوند', 'و', 'یک', 'بار', 'شارژر', 'موبایل', 'ه', 'سوزوند', 'ول', 'با', 'این', 'هیچ', 'مشکل', 'بوجود', 'نیومده', 'تا', 'ال', '.', 'کیفیت', 'خیل', 'خوبه', 'و', 'لامپ', 'ه', 'داره']


Use of words like نیومده , مشکل  is probabily the cause of confusion.

In [1325]:
index=6
print('1:','test_real_outputs',test_real_outputs[wrong_indexes[index]])
print('calculated_out_test',calculated_out_test[wrong_indexes[index]])
print('comment',comment_test[wrong_indexes[index]][1])

1: test_real_outputs 2
calculated_out_test 1
comment ['جنس', 'و', 'چسبندگ', 'عالیه', 'اما', 'کل', 'صفحه', 'رو', 'نمیپوشونه', 'متاسفانه', '.', 'با', 'این', 'دید', 'اگر', 'بخرین', '،', 'ناراض', 'نمیشید', 'از', 'خریدتون', 'مثل', 'من', '.']


Words like عالیه , بخرین  could be the cause of mistake. 